# Tutorial 6. Styling

:::{note}
:icon: false

#### How to style Panel and `panel-material-ui` apps with confidence

In this tutorial you will build intuition for *where* each styling tool applies:

- `styles`: style the outer container of a component
- `stylesheets`: style internals of classic Panel components via CSS
- `sx`: style internals of `panel-material-ui` components
- `theme_config`: define app-wide design rules and component defaults

By the end, you will know which tool to use first, and how to combine them without creating a maintenance mess.

## Learning goals

After this tutorial, you should be able to:

1. Explain the difference between container styling and internal styling.
2. Use `styles` and `stylesheets` for classic Panel widgets.
3. Use `sx` and `theme_config` for `panel-material-ui`.
4. Build a cohesive mini-dashboard with both one-off and global styling.

:::

---

## 1) Mental model: four styling layers

Think of styling in Panel as four layers:

1. **Container layer** (`styles`)  
   You style the box around a component.
2. **Component internals layer (classic Panel)** (`stylesheets`)  
   You style the HTML/CSS inside a widget.
3. **Component internals layer (Material UI)** (`sx`)  
   You style a `panel-material-ui` component using MUI's style system.
4. **System layer** (`theme_config`)  
   You define global theme rules that flow through many components.

If you remember only one thing: **`styles` is outside, `stylesheets`/`sx` are inside, `theme_config` is everywhere.**

---


## 2) Start simple: style the container with `styles`

`styles` applies CSS directly to the component's outer container. It is perfect for spacing, borders, shadows, and backgrounds around a component.

In [ ]:
import panel as pn

pn.extension()

box_style = {
    "background": "#f8fafc",
    "border": "1px solid #dce3eb",
    "border-radius": "10px",
    "padding": "12px",
    "box-shadow": "0 2px 8px rgba(0,0,0,0.08)",
}

pn.widgets.FloatSlider(
    name="Quality score",
    start=0,
    end=100,
    value=55,
    styles=box_style,
)

### Mini Exercise 1 (2 minutes)

Modify the container style so it looks like a warning card:

- warm background color
- left border only
- no shadow

Keep the slider internals unchanged.

In [ ]:
warning_style = {}

pn.widgets.FloatSlider(
    name="Quality score",
    start=0,
    end=100,
    value=55,
    styles=warning_style,
)

:::{note} Solution
:class: dropdown

```python
warning_style = {
    "background": "#fff4e5",
    "border": "none",
    "border-left": "6px solid #f59e0b",
    "border-radius": "8px",
    "padding": "12px",
    "box-shadow": "none",
}

pn.widgets.FloatSlider(
    name="Quality score",
    start=0,
    end=100,
    value=55,
    styles=warning_style,
)
```

:::

## 3) Go deeper: style classic widget internals with `stylesheets`

When you want to change parts *inside* a classic Panel widget (e.g., slider handle, track, labels), use `stylesheets`.

In [ ]:
slider_css = """
:host {
  --slider-size: 6px;
  --handle-width: 18px;
  --handle-height: 18px;
}

.noUi-handle {
  border-radius: 50%;
  box-shadow: none;
  border: 2px solid #0b6bcb;
  background: white;
}

.noUi-connect {
  background: #0b6bcb;
}
"""

pn.widgets.FloatSlider(
    name="Satisfaction",
    start=0,
    end=10,
    value=7,
    stylesheets=[slider_css],
    styles=box_style,
)

Why this is different from `styles`:

- `styles` can change the wrapper box.
- `stylesheets` can target internals like `.noUi-handle` and `.noUi-connect`.


### Mini Exercise 2 (3 minutes)

Create *three* sliders with the same stylesheet but different color accents (`green`, `orange`, `purple`) using `css_classes`.

:::{note} Solution
:class: dropdown

```python
accent_css = """
:host(.green) .noUi-connect, :host(.green) .noUi-handle { border-color: #2e7d32; background: #2e7d32; }
:host(.orange) .noUi-connect, :host(.orange) .noUi-handle { border-color: #ef6c00; background: #ef6c00; }
:host(.purple) .noUi-connect, :host(.purple) .noUi-handle { border-color: #6a1b9a; background: #6a1b9a; }
:host(.green) .noUi-handle, :host(.orange) .noUi-handle, :host(.purple) .noUi-handle {
  box-shadow: none;
  border-radius: 50%;
}
"""

pn.Column(
    *[
        pn.widgets.FloatSlider(
            name=f"{cls.title()} slider",
            start=0,
            end=10,
            value=5,
            css_classes=[cls],
            stylesheets=[accent_css],
        )
        for cls in ("green", "orange", "purple")
    ]
)
```
:::

## 4) `panel-material-ui`: local internal styling with `sx`

For Material UI components, prefer `sx` for one-off or local customizations.

In [ ]:
import panel_material_ui as pmui

pmui.Row(
    pmui.Button(
        label="Primary action",
        button_type="primary",
        sx={
            "textTransform": "none",
            "fontWeight": 700,
            "px": 2,
            "borderRadius": 3,
            "&:hover": {
                "transform": "translateY(-1px)",
                "boxShadow": 3,
            },
        },
    ),
    pmui.Button(
        label="Secondary action",
        variant="outlined",
        sx={"borderStyle": "dashed", "textTransform": "none"},
    ),
)

Use `sx` when:

- the change should affect only one component instance
- you are experimenting quickly
- you need nested selectors on MUI slots (for example `& .MuiSlider-thumb`)

### Mini Exercise 3 (3 minutes)

Style a `pmui.Card` with `sx` so it:

- has a subtle gradient background
- rounds corners more aggressively
- changes shadow on hover

In [ ]:
local_sx = {}

pmui.Card(
    "This card uses local, one-off styling via sx.",
    margin=10,
    title="Styled with sx",
    sx=local_sx
)

:::{note} Solution
:class: dropdown

```python
pmui.Card(
    "This card uses local, one-off styling via sx.",
    title="Styled with sx",
    sx={
        "background": "linear-gradient(135deg, #f5f7fa 0%, #e4ecfb 100%)",
        "borderRadius": 5,
        "transition": "box-shadow 0.2s ease, transform 0.2s ease",
        "&:hover": {
            "boxShadow": 6,
            "transform": "translateY(-2px)",
        },
    },
    margin=10
)
```

:::

## 5) App-wide consistency with `theme_config`

If `sx` is local, `theme_config` is systemic.  
Use it to define your color palette, typography, shape defaults, and component-wide overrides.


In [ ]:
theme = {
    "palette": {
        "primary": {"main": "#6a1b9a"},
        "secondary": {"main": "#1565c0"},
        "success": {"main": "#2e7d32"},
    },
    "shape": {"borderRadius": 12},
    "typography": {
        "fontFamily": ["Inter", "Helvetica Neue", "Arial", "sans-serif"],
        "button": {"textTransform": "none", "fontWeight": 700},
    },
    "components": {
        "MuiButtonBase": {
            "defaultProps": {"disableRipple": True},
        },
        "MuiCard": {
            "styleOverrides": {
                "root": {
                    "border": "1px solid rgba(0,0,0,0.08)",
                }
            }
        },
    },
}

pmui.Column(
    pmui.Button(label="Themed button", button_type="primary"),
    pmui.Card("Cards inherit theme-level overrides.", title="Themed card"),
    theme_config=theme,
    margin=20
)

Notice how multiple children inherit one shared theme. This keeps apps consistent and easier to maintain over time.

### Mini Exercise 4 (4 minutes)

Add dark/light mode-specific theme configs:

- light: vibrant primary color
- dark: calmer primary color
- verify both with a `ThemeToggle`


In [ ]:
theme_by_mode = {
    "light": {},
    "dark": {},
}

pmui.Row(
    "...",
    pmui.ThemeToggle(),
    theme_config=theme_by_mode,
).preview(height=150)

:::{note} Solution
:class: dropdown

```python
theme_by_mode = {
    "light": {
        "palette": {"primary": {"main": "#d81b60"}},
    },
    "dark": {
        "palette": {"primary": {"main": "#9575cd"}},
    },
}

pmui.Row(
    pmui.Button(label="Mode-aware theme", button_type="primary"),
    pmui.ThemeToggle(),
    theme_config=theme_by_mode,
).preview(height=150)
```

:::

## 6) Decision guide: which tool should I use?

- Use `styles` when you want to style the **outside container**.
- Use `stylesheets` when you want to style **internals of classic Panel widgets**.
- Use `sx` when you want to style **internals of one `panel-material-ui` instance**.
- Use `theme_config` when you want **global consistency and inheritance**.

Rule of thumb:

- Start with `theme_config` for design consistency.
- Add `sx` for local exceptions in `panel-material-ui`.
- Use `styles` and `stylesheets` for classic Panel components where needed.

```{mermaid}
flowchart TD
    B{"Is this app-wide consistency across many components?"}

    B -- "Yes" --> T["Use <code>theme_config</code><br/>(palette, typography, defaults, overrides)"]
    B -- "No" --> C{"Is it a <code>panel-material-ui</code> component?"}

    C -- "Yes" --> SX["Use <code>sx</code><br/>(local/internal component styling)"]
    C -- "No" --> D{"Do you want to style outside container or internals?"}

    D -- "Outside container" --> ST["Use <code>styles</code><br/>(wrapper box: spacing, border, background)"]
    D -- "Internals" --> SS["Use <code>stylesheets</code><br/>(CSS vars/selectors for inner parts)"]

    T --> E["Optional local exceptions with <code>sx</code>"]
    SX --> F["Done"]
    ST --> F
    SS --> F
    E --> F
```

## Section Recap Exercise

Build a small page with two columns:

- **Left ("Classic Panel")**: one `pn.widgets.FloatSlider` + one `pn.widgets.TextInput`
  - style container with `styles`
  - style slider internals with `stylesheets`
- **Right ("Material UI")**: one `pmui.FloatSlider` + one `pmui.TextInput` + one `pmui.Button`
  - style one component locally with `sx`
  - apply shared brand identity via parent `theme_config`

### Your constraints

1. Use exactly one `theme_config` dictionary at the parent level.
2. Use at least one `sx` override and explain why it is local.
3. Use both `styles` and `stylesheets` on at least one classic Panel widget.
4. Ensure the app still looks coherent in dark mode.

### Stretch goals

- Add a "Reset style" button that reverts local overrides.
- Add an "Accessibility pass" by increasing contrast and focus visibility.

In [ ]:
import panel as pn
import panel_material_ui as pmui

pn.extension()

app_theme = {
    "light": {},
    "dark": {}
}


pmui.Row(
    pmui.ThemeToggle(),
    theme_config=app_theme,
).preview(height=300)

:::{dropdown} Solution

```python
import panel as pn
import panel_material_ui as pmui

pn.extension()

classic_container = {
    "background": "#f8fafc",
    "border": "1px solid #dce3eb",
    "border-radius": "10px",
    "padding": "10px",
}

classic_css = """
:host { --slider-size: 6px; --handle-width: 16px; --handle-height: 16px; }
.noUi-connect { background: #1565c0; }
.noUi-handle { border-radius: 50%; box-shadow: none; border: 2px solid #1565c0; background: white; }
"""

app_theme = {
    "light": {
        "palette": {"primary": {"main": "#6a1b9a"}, "secondary": {"main": "#1565c0"}},
        "shape": {"borderRadius": 12},
    },
    "dark": {
        "palette": {"primary": {"main": "#9575cd"}, "secondary": {"main": "#64b5f6"}},
        "shape": {"borderRadius": 12},
    },
}

classic_col = pn.Column(
    "### Classic Panel",
    pn.widgets.FloatSlider(name="Classic slider", styles=classic_container, stylesheets=[classic_css]),
    pn.widgets.TextInput(name="Classic input", styles=classic_container),
)

mui_col = pmui.Column(
    "### Material UI",
    pmui.FloatSlider(sx={"& .MuiSlider-thumb": {"borderRadius": 1}}),  # local exception
    pmui.TextInput(label="MUI input"),
    pmui.Button(label="Submit", button_type="primary"),
)

pmui.Row(
    classic_col,
    mui_col,
    pmui.ThemeToggle(),
    theme_config=app_theme,
).preview(height=300)
```

:::

## Recap

You now have a clear styling strategy for Panel apps:

- use `styles` for container-level styling
- use `stylesheets` (classic) or `sx` (MUI) for internal component styling
- use `theme_config` for consistent app-wide design defaults
